In [ ]:

import requests
from bs4 import BeautifulSoup
import json
import os
from glob import glob
from tqdm import tqdm
from utils import *



a = glob('/mnt/tuannb/Crawl_data/Name_domain/*/*.json')
cc = 0
for item in a:
    dt = json.load(open(item, 'r'))
    for itm in dt:
        for im in itm['QA']: 
            if len(im['question']) != 0 and len(im['context']) != 0 and len(im['answer']) != 0:
                cc += 1
            # cc += 1
cc

In [ ]:
url = 'https://thuvienphapluat.vn/phap-luat/trach-nhiem-tra-no-trong-truong-hop-doanh-nghiep-co-khoan-thua-lo-lon-hon-von-dieu-le-cong-ty-co-ph-524378-10544.html'

get_sample(url)

In [ ]:
url = 'https://thuvienphapluat.vn/phap-luat/tieu-chuan-chuc-danh-pho-truong-phong-giao-duc-va-dao-tao-thuoc-uy-ban-nhan-dan-huyen-quy-dinh-nhu--376032-36290.html?rel=goi-y-cung-tag'
def get_all(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        table = soup.find('body').find('div', attrs={'class' : 'tvpl-main container pt-3 pb-3 wap-page-detail'})
        return table
    pass

def get_categories(table):
    list_categires = table.find_all('div', attrs={'class' : 'row'})[1].find('div', attrs={'class' : "col-md-9"}) \
                          .find('article').find('div', attrs={'class' : 'row'}) \
                          .find('section', attrs={'class' : 'col-md-3'}) \
                          .find_all('section', attrs={'class' : 'news-box-keyword mb-3'}) 
    list_cate = []
    for item in list_categires:
        list_cate.append(item.find('header', attrs={'class' : 'fs-6 fw-bold p-2 m-0'}).find('a').text)
    return list_cate

                
get_categories(get_all(url))

### Combine Data

In [ ]:
import json
from glob import glob
from tqdm import tqdm
from utils import *
import json
from glob import glob
from tqdm import tqdm

a = glob('/mnt/tuannb/Crawl_data/Split Phap Luat/*/data_domain.json')
list_folder = json.load(open('/mnt/tuannb/Crawl_data/Split Phap Luat/doanh nghiep/data_domain.json', 'r'))


total_data = []
q = 0

for item in tqdm(a):
    with open(item, 'r') as f:
        data = json.load(f)
    print(item.split('/')[-2])
    new_data, leng = get_data_process_and_combine(data)
    total_data += new_data
    q += leng
    with open('/'.join(item.split('/')[:-1]) + '/combine_data.json', 'w', encoding='utf-8') as f:
        json.dump(new_data, f, ensure_ascii=False)
print('total: ' + str(q))


a = glob('/mnt/tuannb/Crawl_data/Split Phap Luat/*/combine_data.json')

count_length = [0 for _ in range(10000)]
for item in a:
    with open(item, 'r') as f:
        data = json.load(f)
    for sample in tqdm(data):
        for item in sample['samples']:
            count_length[len(item['context'].split(' '))] += 1
        

In [ ]:
import matplotlib.pyplot as plt

max_index = -1  # Đặt giá trị ban đầu là -1 để xác định rằng chưa tìm thấy phần tử lớn nhất
total = 0
plt.plot(count_length[:1000])
for num in count_length[:1001]:
    total += num

# Tính trung bình
average = total / 1000

# Đặt nhãn cho trục x và trục y (tuỳ chọn)
plt.xlabel('Length Context')
plt.ylabel('Count Context')

# Đặt tiêu đề cho biểu đồ (tuỳ chọn)
plt.title('Mean: ' + str(average))

# Hiển thị biểu đồ
plt.show()

In [ ]:
import matplotlib.pyplot as plt

max_index = -1  # Đặt giá trị ban đầu là -1 để xác định rằng chưa tìm thấy phần tử lớn nhất
total = 0
count = 0
plt.plot(count_length[:1000])
for idx, num in enumerate(count_length[:1001]):
    total += idx*num
    count += num
    
# Tính trung bình
average = total / count

# Đặt nhãn cho trục x và trục y (tuỳ chọn)
plt.xlabel('Length Context')
plt.ylabel('Count Context')

# Đặt tiêu đề cho biểu đồ (tuỳ chọn)
plt.title('Mean: ' + str(average))


# Hiển thị biểu đồ
plt.show()

In [ ]:
new_data = []

for item in total_data:
    for itm in item['samples']:
        new_data.append(itm)

In [ ]:
len(new_data)

In [ ]:
data = []

remove_answer = open('/mnt/tuannb/Crawl_data/Remove/remove_answer.txt', 'r').read().split('\n')

for item in tqdm(new_data):
    flag = True
    for remove in remove_answer:
        if remove in item['answer']:
            flag = False
            break
        
    if flag == True:
        data.append(item)
        
len(data)

In [ ]:
def capitalize_first_letter(input_string):
    """Capitalizes the first letter of a string."""
    if not input_string:
        return input_string
    
    return input_string[0].upper() + input_string[1:]

import re

def format_text_advanced(text):
    """
    Format the entire text by removing leading/trailing whitespaces and special characters,
    capitalize the first letter of each sentence, handle line breaks after commas, and remove line breaks before commas.
    
    Parameters:
        text (str): The input text.
    
    Returns:
        str: Formatted text.
    """
    # Remove line breaks before commas
    text = re.sub(r'\n\s*,', ',', text)

    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Format each sentence
    formatted_sentences = []
    for i, sentence in enumerate(sentences):
        formatted_sentence = re.sub(r'^[^\w]+', '', sentence.strip()).capitalize()

        # Check if the current sentence ends with a period and the next sentence starts with a comma
        if i < len(sentences) - 1 and sentence.endswith('.') and sentences[i + 1].startswith(','):
            formatted_sentence += ','

        formatted_sentences.append(formatted_sentence)

    # Join the formatted sentences into formatted text
    formatted_text = ' '.join(formatted_sentences)
    return formatted_text


def all_format(text):
    return capitalize_first_letter(capitalize_first_letter((text.removeprefix('\n').removeprefix(',').strip().removeprefix(',').strip().replace('\n,', ','))))


dt = []

for item in tqdm(data):
    dt.append({
        'question': all_format(item['question']),
        'context': all_format(item['context']),
        'answer': all_format(item['answer']).strip(),
        'reference': item['reference'],
    })

In [ ]:

class TerminalColors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    OKPINK = "\033[95m"
    FAIL = '\033[91m'
    ENDC = '\033[0m'  # Kết thúc màu sắc
    

vl = []
for itm in dt:
    if 'Quảng cáo phân bón được quy định như thế nào?' in itm['question'] or 'Tiêu chuẩn để chăn nuôi nông hộ là gì?' in itm['question']:
        # print(TerminalColors.OKGREEN + itm['question'] + "\n" + TerminalColors.ENDC)
        # print(TerminalColors.WARNING + itm['answer']  + '\n' + TerminalColors.ENDC)
        # print('---'*40)
        continue
    else:
        vl.append(itm)

In [ ]:
json.dump(vl, open('/mnt/tuannb/Data/Law_QA_4_12/law_qa.json', 'w'), ensure_ascii=False)

In [ ]:
import random

pseudo = []

for _ in range(40000):
    a = random.randint(0, 170000)
    context = ''
    while True:
        b = random.randint(0, 170000)
        if vl[a]['question'] != vl['']

In [1]:
import json

split_domain = json.load(open('/mnt/tuannb/Data/Law_QA_4_12/split_domain.json', 'r'))

import random
from tqdm import tqdm

pseudo = []

list_domain = list(split_domain.keys())

for domain in tqdm(list_domain):
    for _ in range(2000):
        length = len(split_domain[domain])
        first = random.randint(0, length - 2)
        second = random.randint(0, length - 2)
        if split_domain[domain][first]['answer'] != split_domain[domain][second]['answer']:
            pseudo.append({
                'question': split_domain[domain][first]['question'],
                'context': split_domain[domain][second]['context'],
                'answer': 'Không có thông tin',
                'reference': split_domain[domain][first]['reference'],
                'domain': split_domain[domain][first]['domain']
            })

len(pseudo)

100%|██████████| 27/27 [00:00<00:00, 94.57it/s]


53977

In [2]:
total = []

data = []
for domain in tqdm(list_domain):
    for item in split_domain[domain]:
        data.append(item)
        
random.shuffle(pseudo)
random.shuffle(data)

train = []
for item in data[:160000]:
    train.append(item)
    
for item in pseudo[:40000]:
    train.append(item)

test = data[160000:]
pseudo_test = pseudo[40000:]
random.shuffle(train)
# json.dump(train, open('/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/train.json', 'w'), ensure_ascii=False)
# json.dump(test, open('/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/test.json', 'w'), ensure_ascii=False)
# json.dump(pseudo_test, open('/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/pseudo_test.json', 'w'), ensure_ascii=False)
    
# json.dump(pseudo, open('/mnt/tuannb/Data/Law_QA_4_12/pseudo.json', 'w'), ensure_ascii=False)


100%|██████████| 27/27 [00:00<00:00, 2126.89it/s]


In [15]:
import json

def save_jsonl(data, path):
    with open(path, "w", encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")   
    pass

def load_jsonl(path_file):
    list_data = []
    with open(path_file, 'r') as file:
        for line in file:
            list_data.append(json.loads(line))
    return list_data

# save_jsonl(train, path='/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/train.jsonl')

# save_jsonl(test, path='/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/test.jsonl')

# save_jsonl(pseudo_test, path='/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/pseudo_test.jsonl')

In [19]:
x = load_jsonl('/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/train_v2.jsonl')
len(x)

138714

In [20]:
x[1]

{'question': 'Công chức khi sử dụng hồ sơ điện tử cá nhân công chức trên Hệ thống thông tin công chức viên chức Bộ Tài nguyên và Môi trường cần tuân theo những quy định gì?',
 'context': 'Căn cứ tại khoản 1 Điều 9 Quy chế quản lý, khai thác Hệ thống thông tin công chức, viên chức Bộ Tài nguyên và Môi trường ban hành kèm theo Quyết định 2715/QĐ-BTNMT năm 2016, có quy định về nghiên cứu, khai thác và sử dụng thông tin hồ sơ điện tử cá nhân như sau:\nNghiên cứu, khai thác và sử dụng thông tin hồ sơ điện tử cá nhân\n1. Đối tượng được nghiên cứu, sử dụng và khai thác hồ sơ điện tử cá nhân:\na) Đơn vị quản lý, sử dụng công chức, viên chức được nghiên cứu, khai thác và sử dụng thông tin hồ sơ điện tử công chức, viên chức trên hệ thống để phục vụ yêu cầu công tác.\nb) Công chức, viên chức được sử dụng toàn bộ thông tin hồ sơ điện tử cá nhân của mình nhưng không được phép sửa chữa, thêm bớt nội dung thông tin hồ sơ điện tử cá nhân của mình.\n… \nCăn cứ tại khoản 2 Điều 9 Quy chế quản lý, khai t

In [5]:
from datasets import load_dataset
from tqdm import tqdm

train = load_dataset('json', data_files='/mnt/tuannb/Data/Law_QA_4_12/Train_Version_1/train_full.jsonl')
count = 0

for item in tqdm(train['train']):
    if len(item['input_ids']) > 1024:
        count += 1
        
count


100%|██████████| 200000/200000 [01:52<00:00, 1776.60it/s]


0